In [1]:
from importlib import reload
import os
import sys
import platform
import re
import datetime
import multiprocessing
from glob import glob
import logging
from tracker.tank import Tank
from tracker.tracker import Tracker
from tracker import utils
import cv2
from converter import Converter

import tracker
reload(tracker.utils)
from tracker import utils
reload(tracker.tank)
from tracker.tank import Tank
reload(tracker.tracker)
from tracker.tracker import Tracker

In [2]:
''' Select input files to track and an output directory for each. '''

# tracking_dir = '../tracking/full_20-10-06'
# tracking_dir = '../tracking/test_20-10-04'
tracking_dir = 'output_data2'

# input_files = sorted(glob('../raw_videos/*.avi')) #[:2]
# input_files = sorted(glob('../raw_videos/Pa_*_*4dpf_*.avi')) #[:2]
# input_files = [ '../raw_videos/Pa_Fri_84dpf_GroupA_n2b_20200821_1304.avi' ]
# input_files = [ 'input_data/Pa_Fri_84dpf_GroupC_n5_20200821_1450.avi' ]
# input_files = [ '../raw_videos/Pa_Fri_84dpf_GroupC_n5_20200821_1450.avi' ]
input_files = [ '../raw_videos/SF_Sat_14dpf_GroupA_n5_2020-06-13-120445-0000.avi' ]

input_dict = {}
for input_file in input_files:
    d,fn       = os.path.split(input_file)
    fn,ext     = os.path.splitext(fn)
    output_dir = os.path.join(tracking_dir,fn)
    n_ind      = int(re.findall('_n(\d+)[^\d]*_',fn)[0])
    
    # If a repaired version exists, use that instead.
    repaired_file = os.path.join(d,'repaired',fn+'-repaired'+ext)
    input_file = repaired_file if os.path.exists(repaired_file) else input_file
    
#     # If a trial file already exists, skip.
#     if os.path.exists(os.path.join(output_dir,'trial.pik')):
#         continue
        
    input_dict[input_file] = dict( output_dir=output_dir, n_ind=n_ind )

print(len(input_dict))

# for f,D in input_dict.items():
#     print(f)
#     print('   ',D)

1


In [3]:
import tracker
reload(tracker.utils)
from tracker import utils
reload(tracker.tank)
from tracker.tank import Tank
reload(tracker.tracker)
from tracker.tracker import Tracker


''' Create output directories. Compute backgrounds. '''

settings = dict(
    threshold_offset = 13,    # threshold offset for contour-finding
    # Background subtraction (for naive background subtraction only).
    bkgSub_options   = dict( n_training_frames = 100, # number of frames used to compute background
                             t_start = 0, t_end = -1, # time range used to compute background
                             contrast_factor = 4,     # post-subtraction contrast enhancement factor
                             secondary_subtraction = True )
    )

for input_file,D in input_dict.items():
    tracker = Tracker(input_video=input_file, **D)
    
    # Create output directory.
    tracker.init_directory()
    
    # Create a link to the input video in the output directory.
    output_dir = D['output_dir']
    input_shortcut = os.path.join(output_dir,'raw'+ext)
#     if not 'windows' in platform.system().lower() and not os.path.exists(input_shortcut):
    try:
        os.symlink(os.path.relpath(input_file,output_dir),input_shortcut)
    except:
        pass
        
    # Detect background.
    tracker.init_video_input()
    tracker.bkgSub_options   = settings['bkgSub_options']
    tracker.offset           = settings['threshold_offset']
    tracker.init_background()
    tracker.init_secondary_background()
    tracker.release()

In [4]:
''' Trace tanks. '''

for input_file,D in input_dict.items():
    tracker = Tracker(input_video=input_file, **D)
    tracker.init_background()
    tracker.release()
    tracker.init_tank(thresh=8,dilate=15)

In [5]:
cv2.destroyAllWindows()

In [6]:
import tracker
reload(tracker.utils)
from tracker import utils
reload(tracker.tank)
from tracker.tank import Tank
reload(tracker.tracker)
from tracker.tracker import Tracker

settings.update(dict(
    t_start          = 300,     # Time at which to start tracking, in seconds.
    t_end            = 305,    # Time at which to end tracking, in seconds.
    
    # Contour detection.
    n_pixel_blur     =  7,    # square-root of n-pixels for threshold blurring
    block_size       = 15,    # contour block size
    threshold_offset = 13,    # threshold offset for contour-finding
    min_area         = 30,    # minimum area for detection
    max_area         = 400,   # maximum area for detection
    ideal_area       = 100,   # ideal area to rank contours in first frame (default=(min_area+max_area)/2)
    max_aspect       = 15,    # maximum aspect ratio for detection
    ideal_aspect     = 3,     # ideal aspect ratio to rank contours in first frame (default=max_aspect/2)
    area_penalty     = 0.5,   # weight of area change when connecting fish across frames
    n_extra          = 2,     # number of extra contours to keep track of
    morph_transform  = [], 
#     morph_transform  = [(cv2.MORPH_OPEN,2),(cv2.MORPH_CLOSE,2)], 
#     morph_transform  = [(cv2.MORPH_CLOSE,20),(cv2.MORPH_OPEN,3)], 
#     morph_transform  = [(cv2.MORPH_DILATE,5),(cv2.MORPH_OPEN,8)], 
                              # sequence of morphological operations to perform
                              # on thresholded image before extracting contours.
    reversal_threshold = 0.5, # average frame-to-frame displacement against the director 
                              # over the last few frames to trigger a reversal. 
    live_preview     = False, # Toggle live preview of tracking.

    # What information to draw on the tracking output video.
    video_output_options = dict( tank=True, points=False, directors=True, 
                                 extra_points=True, timestamp=True, 
                                 contours=True, contour_color=(100,255,0), 
                                 contour_thickness=1 )
    ))

def track(input_file):
    
    globals().update(settings)
    globals().update(input_dict[input_file])
    video_output_options = settings['video_output_options']
    
    utils.add_log_file(os.path.join(output_dir,'log.txt'))
    logging.info(utils.parindent+'Initializing')
    tracker = Tracker( input_video=input_file, **input_dict[input_file], **settings )
    tracker.init_all()
    tracker.init_secondary_background()
    tracker.save_settings()

    frames_dir = os.path.join(output_dir,'frames')

    try:
        tracker.set_frame(tracker.frame_start)
        tracker.init_live_preview()
        n_report = max(1,int((tracker.frame_end-tracker.frame_start)/50))
        for i_frame in range(tracker.frame_start, tracker.frame_end+1):
            if i_frame%n_report==0:
                percent = (i_frame-tracker.frame_start)/(tracker.frame_end-tracker.frame_start)
                logging.info(utils.parindent + f'Tracking: {tracker.get_current_timestamp()}, ' +
                             f'{tracker.get_percent_complete():4.1f}% complete')
            if tracker.get_next_frame():
                # When b evaluates to True, intermediate tracking steps are saved 
                # as images in "[output_dir]/frames".
                b = False # True # 260<tracker.frame_num<275 # 
                if b:
                    if not os.path.exists(frames_dir):
                        os.mkdir(frames_dir)
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-1_raw.png'),tracker.frame)
                tracker.subtract_background()
                if b:
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-2a_bkg-subtracted.png'),tracker.frame)
                tracker.subtract_secondary_background()
                if b:
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-2b_bkg2-subtracted.png'),tracker.frame)
                tracker.mask_tank()
                tracker.detect_contours()
                tracker.connect_frames()
                if b:
#                     cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-2_masked.png'),tracker.frame)
                    video_output_options.update(directors=False)
                    tracker.draw(**video_output_options)
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-3_contours.png'),tracker.frame)
                    video_output_options.update(directors=True)
                tracker.draw(**video_output_options)
                if b:
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-4_directors.png'),tracker.frame)
                tracker.write_frame()
                if not tracker.post_frame(delay=1):
                    break
        tracker.release()
        logging.info(utils.parindent+'Saving')
        tracker.save_trial()
        logging.info(utils.parindent+'Done')

    except:
        tracker.release()
        logging.info('Failed')
        for info in sys.exc_info():
            logging.warning(info)
        %tb

    return


# Multi-threading doesn't work in Windows. If the OS is Windows, set n_thread to 1.
n_threads = 1 if 'windows' in platform.system().lower() else 1

if n_threads==1:
    
    for input_file in input_dict.keys():
        utils.reset_logging()
        utils.add_log_stream(sys.stdout)
        logging.info(input_file)
        handler = logging.root.handlers[-1]
        handler.terminator = ''
        handler.setFormatter(logging.Formatter('\r'+' '*200+'\r     %(message)s'))
        track(input_file)
        break
    
else:
    def track_(k):
        utils.reset_logging()
        return track(k)
    pool = multiprocessing.Pool(n_threads)
    pool.map(track_,input_dict.keys())

../raw_videos/SF_Sat_14dpf_GroupA_n5_2020-06-13-120445-0000.avi
     Done                                                                                                                                                                                               

In [ ]:
input_dict.keys()

In [ ]:
input_file,D = next(iter(input_dict.items()))
trial_file   = Tracker(input_video=input_file, **D).trial_file
with open(trial_file,'rb') as f:
    trial = pickle.load(f)
print(trial.keys())

xy = trial['data'][:,:trial['n_ind'],:2]
xy[:,0,:].shape
for i in range(xy.shape[1]):
    plt.plot(*xy[:,i,:].T,label=f'{i}')
plt.legend(loc=(1.05,0.5))
plt.show()

In [ ]:
import tracker
reload(tracker)
import tracker.analyze as analyze
reload(tracker.analyze)

input_file,D = next(iter(input_dict.items()))
trial_file   = Tracker(input_video=input_file, **D).trial_file
trial        = analyze.load_trial(trial_file)
globals().update(trial)

for i in range(n_ind):
    plt.plot(df[(i,'x_px')],df[(i,'y_px')],label=f'{i}')
plt.legend(loc=(1.05,0.5))
plt.show()

In [ ]:
for input_file,D in input_dict.items():
    
    tracker = Tracker(input_video=input_file, **D)
    with open(tracker.trial_file,'rb') as f:
        trial = pickle.load(f)

    # area
    plt.plot(trial['data'][:,:,3])
    plt.ylim(0,None)
    plt.show()
    
    # aspect ratio
    plt.plot(trial['data'][:,:,4])
    plt.ylim(1,None)
    plt.show()
    break